#### Imports

In [3]:
import pandas as pd
import random
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#### Data Preprocessing and Cleaning

In [4]:
df = pd.read_csv('application_train_clean.csv')
df = df.drop(['Unnamed: 0','SK_ID_CURR'],axis=1)
print(df.columns[df.dtypes == object])
df = pd.get_dummies(df, columns = df.select_dtypes(include=['object']).columns, drop_first=True, dtype=int)
df.to_csv('cleaned_train.csv', index=False)
df.head()


Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'ORGANIZATION_TYPE'],
      dtype='object')


,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648,...,0,0,0,0,0,0,0,0,0,0
1,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186,...,0,0,0,0,0,0,0,0,0,0
2,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260,...,0,0,0,0,0,0,0,0,0,0
3,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311,...,0,0,0,0,0,0,0,0,0,0
4,0,0,99000.0,490495.5,27517.5,454500.0,0.035792,-16941,-1588,-4970,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X, y = df.drop('TARGET',axis=1), df['TARGET']

### Cross-Validation Splits

#### Stratified Split

In [6]:
# Stratified Split
def stratified_split(data, test_size=0.1, validation_size = 0.1):

    class_1 = data[data['TARGET'] == 1]
    class_0 = data[data['TARGET'] == 0]

    test_count_1 = int(len(class_1) * test_size)
    validation_count_1 = int(len(class_1) * validation_size)
    test_count_0 = int(len(class_0) * test_size)
    validation_count_0 = int(len(class_0) * validation_size)

    class_1 = class_1.sample(frac=1)
    class_0 = class_0.sample(frac=1)

    # Split each class into test, validation, and train sets
    test_data = pd.concat([class_1.iloc[:test_count_1], class_0.iloc[:test_count_0]])
    validation_data = pd.concat([
        class_1.iloc[test_count_1:test_count_1 + validation_count_1],
        class_0.iloc[test_count_0:test_count_0 + validation_count_0]
    ])
    train_data = pd.concat([
        class_1.iloc[test_count_1 + validation_count_1:],
        class_0.iloc[test_count_0 + validation_count_0:]
    ])

    # Split features and target for each set
    X_train = train_data.drop('TARGET',axis=1)
    X_test = test_data.drop('TARGET',axis=1)
    X_val = validation_data.drop('TARGET',axis=1)
    y_train = train_data['TARGET']
    y_test = test_data['TARGET']
    y_val = validation_data['TARGET']

    return X_train, X_test, X_val, y_train, y_test, y_val

In [7]:
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
y_val.head

<bound method NDFrame.head of 153490    1
178483    1
41261     1
33236     1
73006     1
         ..
123583    0
214837    0
176260    0
60397     0
68819     0
Name: TARGET, Length: 26341, dtype: int64>

#### Custom Split

In [8]:
# Custom Split

def custom_split(data, valid_prop = 0.1, test_prop = 0.1, random_seed = 1738):

    train_prop = 1 - valid_prop - test_prop

    # define bins for age and income
    age_bins = [0, 40, 60, np.inf]
    age_labels = ['young', 'middle_aged', 'senior']
    income_bins = [0, 30000, 70000, np.inf]
    income_labels = ['low', 'medium', 'high']

    # convert age to years
    data['age'] = data['DAYS_BIRTH']/-365

    # create binned variables
    data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels)
    data['income_group'] = pd.cut(data['AMT_INCOME_TOTAL'], bins=income_bins, labels=income_labels)

    # create a key for each group (combination of gender, age, and income)
    data['group_key'] = data['CODE_GENDER_M'].astype(str) + '_' + data['age_group'].astype(str) + '_' + data['income_group'].astype(str)

    # shuffle the data
    data = data.sample(frac=1, random_state=random_seed).reset_index(drop=True)

    # split the data based on key
    train_data = pd.DataFrame()
    val_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for key, group in data.group_by('group_key'):
        n = len(group)
        n_train = int(n * train_prop)
        n_val = int(n * valid_prop)

        train_data = pd.concat([train_data, group[:n_train]])
        val_data = pd.concat([val_data, group[n_train:n_train + n_val]])
        test_data = pd.concat([test_data, group[n_train + n_val:]])

    # drop all unnecesary columns
    train_data = train_data.drop(columns=['age','age_group', 'income_group', 'strat_key'])
    val_data = val_data.drop(columns=['age','age_group', 'income_group', 'strat_key'])
    test_data = test_data.drop(columns=['age','age_group', 'income_group', 'strat_key'])

    X_train = train_data.drop(['TARGET'])
    y_train = train_data['TARGET']

    X_validation = val_data.drop(['TARGET'])
    y_validation = val_data['TARGET']

    X_test = test_data.drop(['TARGET'])
    y_test = test_data['TARGET']

    return X_train, y_train, X_validation, y_validation, X_test, y_test


#### Random Split

In [9]:
def random_train_test_split(X, y, test_size=0.2, val_size=0.2, random_seed=42):
    np.random.seed(random_seed)
    
    n = len(X)
    indices = np.arange(n)
    np.random.shuffle(indices)
    
    test_indices = indices[:int(n * test_size)]
    val_indices = indices[int(n * test_size):int(n * (test_size + val_size))]
    train_indices = indices[int(n * (test_size + val_size)):]
    
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    X_val, y_val = X[val_indices], y[val_indices]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

### Metrics

#### Accuracy

In [10]:
# Accuracy
def compute_accuracy(truth, predicted):
    return np.mean(truth == predicted)


#### F1 Score

In [11]:
# F1 Score
def calculate_f1(y_true, y_pred):
    TP = FP = TN = FN = 0

    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            TP += 1
        elif true == 0 and pred == 1:
            FP += 1
        elif true == 0 and pred == 0:
            TN += 1
        elif true == 1 and pred == 0:
            FN += 1  ,

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    if precision + recall == 0:
        return 0

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

#### ROC-AUC

In [12]:
def calc_roc_auc(y_true, y_prob):
  sorted_indices = np.argsort(y_prob)
  true_sort = y_true[sorted_indices]
  prob_sort = y_prob[sorted_indices]

  TP = np.cumsum(true_sort)
  FP = np.cumsum(1 - true_sort)

  TPR = TP / TP[-1]
  FPR = FP / FP[-1]

  # rocauc
  rocauc_score = np.trapz(FPR, TPR)

  return rocauc_score

### Models

#### Logistic Regression W/ Penalization

In [13]:
# logistic regression w penalty
lr_penalty = LogisticRegression(penalty='l1', C=1.0)
lr_penalty.fit(X, y)
w = lr_penalty.coef_[0]
b = lr_penalty.intercept_[0]
print(f'intercept: {b} \n weights: {w}')

ValueError: Solver lbfgs supports only 'l2' or None penalties, got l1 penalty.

#### SVC

In [ ]:
# SVC Model
#SVC
svc_model = SVC(gamma='auto', kernel='linear') #Can add standard scaler if you want to scale, but left it out to interpret results
svc_model.fit(X, y)
weights = svc_model.coef_
b = svc_model.intercept_
print(f'weights: {weights} \n constant: {b}')
print(f'Actual decision boundary: {-b/weights}')


#### LDA


In [15]:
# LDA Model

lda_model = LinearDiscriminantAnalysis()

lda_model.fit(X,y)

weights = lda_model.coef_
b = lda_model.intercept_

print(f'weights: {weights} \n constant: {b}')

weights: [[-6.99163540e-03  1.31726591e-07  2.51570304e-06  7.90656431e-06
  -2.97870729e-06  3.30432518e+00  3.10964090e-05  6.09622973e-05
   1.22852234e-05  6.92994698e-05 -5.49536286e-14 -1.13795177e+00
   2.32766005e-01 -1.04844708e-01 -6.25567688e-02 -3.06116334e-02
   1.04697501e-02 -1.36067968e-01  3.02887942e-01  2.76813800e-03
  -2.55896995e-01  1.49182311e-01 -1.99011053e-01  3.04084734e-01
   3.83162041e-03  3.11606065e-02 -2.58909916e+00  2.51117890e-02
   1.66423113e-01 -2.74171808e-02  6.51061324e-02  9.02112208e-05
  -1.89972739e-13  5.84966274e-01 -2.68696131e-01  5.20468085e-01
   5.29385595e-01  1.62834989e-01  3.62887028e-01  3.65029890e-01
  -1.06538072e+00 -1.13021294e-01  2.83997546e-01 -4.60871568e-01
  -5.49975143e-01 -5.08995001e-01 -4.66557311e-01 -1.02500534e+00
  -4.28563348e-01 -2.19961398e-01  5.62758668e-01  4.62208840e-01
  -2.76730968e-02  1.31825353e-01 -1.53816093e-02 -3.96546600e-03
  -1.66288010e-02  3.37600839e-02  2.41319189e-01  4.22802912e-01
 